# Dataset notebook

Import all the different chunks of the dataset, put into a single file and remove unneeded columns (first two)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('openfood_dataset.tsv', sep='\t', low_memory=False)

## Extract just the column needed

- product_name
- fat_100g
- carbohydrates_100g
- proteins_100g
- energy_100g (kJ)
- saturated_fat_100g
- sugars_100g
- fiber_100g
- salt_100g

In [3]:
COLUMNS = ["product_name", "fat_100g", "carbohydrates_100g", "proteins_100g", "energy_100g", "saturated-fat_100g", "sugars_100g", "fiber_100g", "salt_100g"]

In [4]:
df_projected = df[COLUMNS]
df_projected.columns = ['product name', 'fat', 'carbohydrates', 'proteins', 'calories', 'saturated fat', 'sugars', 'fiber', 'salt']

In [5]:
df_projected

,product name,fat,carbohydrates,proteins,calories,saturated fat,sugars,fiber,salt
0,Farine de blé noir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Banana Chips Sweetened (Whole),28.57,64.29,3.57,2243.0,28.57,14.29,3.6,0.00000
2,Peanuts,17.86,60.71,17.86,1941.0,0.00,17.86,7.1,0.63500
3,Organic Salted Nut Mix,57.14,17.86,17.86,2540.0,5.36,3.57,7.1,1.22428
4,Organic Polenta,1.43,77.14,8.57,1552.0,NaN,NaN,5.7,NaN
...,...,...,...,...,...,...,...,...,...
356022,"Mint Melange Tea A Blend Of Peppermint, Lemon ...",0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00000
356023,乐吧泡菜味薯片,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356024,Biscottes bio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356025,Tomates aux Vermicelles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sanitize dataset

It seems like some rows are not following the rule of values per 100g of product, since some foods have values for either protein, fat or carbs above 100. 
For this reason, those rules are pruned from the dataset.

Dataset is then sorted aphabetically by food name.

In [6]:
df_without_nulls = df_projected.dropna(how='any',axis=0) 

In [7]:
df_without_nulls

,product name,fat,carbohydrates,proteins,calories,saturated fat,sugars,fiber,salt
1,Banana Chips Sweetened (Whole),28.57,64.29,3.57,2243.0,28.57,14.29,3.6,0.00000
2,Peanuts,17.86,60.71,17.86,1941.0,0.00,17.86,7.1,0.63500
3,Organic Salted Nut Mix,57.14,17.86,17.86,2540.0,5.36,3.57,7.1,1.22428
7,Organic Muesli,18.75,57.81,14.06,1833.0,4.69,15.62,9.4,0.13970
12,Zen Party Mix,36.67,36.67,16.67,2230.0,5.00,3.33,6.7,1.60782
...,...,...,...,...,...,...,...,...,...
355981,"Biscuits aux céréales, aux pépites de chocolat...",22.00,60.00,8.00,2008.0,9.00,27.00,5.0,0.28000
355985,Natural Cassava,0.00,87.06,1.18,1477.0,0.00,2.35,4.7,0.03048
356005,Tartines craquantes bio au sarrasin,2.80,74.80,13.00,1643.0,0.60,2.60,5.9,0.68000
356017,Thé vert Earl grey,0.20,0.50,0.50,21.0,0.20,0.50,0.2,0.02540


In [8]:
df_sanitized = df_without_nulls[(df_without_nulls["proteins"] < 100) | (df_without_nulls["carbohydrates"] < 100) | (df_without_nulls["fat"] < 100)]
df_sanitized = df_sanitized.drop(df_sanitized[(df_sanitized['proteins'] == 0.0) & (df_sanitized['carbohydrates'] == 0.0) & (df_sanitized['fat'] == 0.0)].index)


## Convert kJ to calories

In [9]:
df_sanitized['calories'] = df_sanitized['calories'].apply(lambda energy: round(float(energy) * 0.239006, 2))

## Remove duplicates

In [19]:
df_no_duplicates = df_sanitized.drop_duplicates('product name')

## Sort by name and save final dataset

In [20]:
final_dataset = df_no_duplicates.sort_values('product name')

In [21]:
final_dataset.to_csv("openfood_sanitized_dataset.csv", index=False)